## ChatBOT with PyTorch
- ChatBOT's training data contains sequences having different **tags** such as greeting, goodbye...
- Under each tag, there are 2 main parts: **patterns** and **responses**.
- On vectorization step, the concept of **BOW (Bag Of Words)** is used. 
- Each pattern is represented via a one-hot-vector with size of the whole corpus.

    - For example: let's say all words are: ["Hi","How","are","you","bye","see","later"] 
    - "Hi" --> [1,0,0,0,0,0,0] and its label is 0 (greeting)
    - "See you later" --> [0,0,0,1,0,1,1] and its label is 1 (goodbye)

- Technique of **Tokenization** is used via **NLTK (Natural Language ToolKit)** for splitting a string into meaningful (not always tho) units.
- And technique of **Stemming** is used to generate root form of the words. i.e. crude heuristic that chops of the ends off of words.
### -----------------------------------------------------------------------------------------------------------
### NLP Preprocessing Pipeline
- A basic preprocessing pipeline used for extracting ChatBOT training data contains these steps:
    1) Tokenize
    2) Lower + Stem
    3) Exclude punctuation characters
    4) Bag of words
- When an input sequence (string) goes through these steps, one-hot-vector for training (X) is obtained.
    - For example: input sequence "Is anyone there?" 
    - after tokenization --> ["Is","anyone","there","?"]
    - after lowering and stemming --> ["is","anyon","there","?"]
    - after excluding punctuations --> ["is","anyon","there"]
    - after BOW --> [0,0,0,1,0,1,0,1] is our input vector X

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import nltk_utils, train_utils
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name(device)

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

###  Dataset Implementation

In [2]:
# Hyperparameters
batch_size = 8
num_workers = 2
learning_rate = 0.001
n_epochs = 1000

In [3]:
X_train, y_train, all_words, tags = train_utils.get_train_data()

class ChatBOTDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
dataset = ChatBOTDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

the tags: ['allergen_information', 'cafe_environment', 'customer_service', 'customization', 'daily_specials', 'delivery', 'drinks', 'events', 'food_options', 'funny', 'goodbye', 'greeting', 'hours', 'items', 'location', 'loyalty_program', 'payments', 'prices', 'specialty_drinks', 'sustainability_practices', 'thanks', 'wifi']
number of tags: 22


### Model Implementation

In [4]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)
        return out
    
input_size = len(X_train[0])
hidden_size = 32
output_size = len(tags)

### Training

In [5]:
model = NeuralNet(input_size, hidden_size, output_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(n_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)
        loss_value = criterion(outputs, labels)

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'epoch {epoch+1}/{n_epochs}, loss={loss_value.item():.4f}')
        
print(f'Final Loss: {loss_value.item():.4f}')

epoch 100/1000, loss=0.0036
epoch 200/1000, loss=0.0006
epoch 300/1000, loss=0.0003
epoch 400/1000, loss=0.0001
epoch 500/1000, loss=0.0002
epoch 600/1000, loss=0.0000
epoch 700/1000, loss=0.0000
epoch 800/1000, loss=0.0000
epoch 900/1000, loss=0.0000
epoch 1000/1000, loss=0.0000
Final Loss: 0.0000


In [6]:
chatBOT_model = {
    "model_state_dict": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}
FILEPATH = "chatBOT_model.pth"
torch.save(chatBOT_model,FILEPATH)